In [ ]:
pip install tabulate

In [1]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tabulate import tabulate

# Load the datasets
data_path = 'CLEAN- PCOS SURVEY SPREADSHEET.csv'
df = pd.read_csv(data_path)

recipes_path = 'recipes.csv - Sheet1.csv'
recipes_df = pd.read_csv(recipes_path, encoding='latin-1')

exercise_path = 'exercise.csv - sheet1.csv'
exercise_df = pd.read_csv(exercise_path, encoding='latin-1')

# Fill missing values
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Define target column and features
target_column = 'Have you been diagnosed with PCOS/PCOD?'  # Adjust this as needed
X = df_imputed.drop(columns=[target_column])
y = df_imputed[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

def generate_diet_and_exercise_plan():
    days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    meals = ["Breakfast", "Lunch", "Dinner"]

    diet_table_data = [["Days", "Breakfast", "Lunch", "Dinner"]]
    exercise_table_data = [["Days", "Morning Exercise", "Evening Exercise"]]

    recipe_name_column = recipes_df.columns[0]
    exercise_name_column = exercise_df.columns[0]

    for day in days:
        diet_row = [day]
        for meal in meals:
            recipe = recipes_df.sample(1)
            diet_row.append(recipe[recipe_name_column].values[0])
        diet_table_data.append(diet_row)

        exercise_row = [day]
        morning_exercise = exercise_df.sample(1)
        evening_exercise = exercise_df.sample(1)
        exercise_row.extend([morning_exercise[exercise_name_column].values[0], evening_exercise[exercise_name_column].values[0]])
        exercise_table_data.append(exercise_row)

    print("\n--- Personalized Weekly Diet Plan ---")
    print(tabulate(diet_table_data, headers="firstrow", tablefmt="grid"))
    print("\n--- Personalized Weekly Exercise Plan ---")
    print(tabulate(exercise_table_data, headers="firstrow", tablefmt="grid"))
    print("\nPlease follow the meal and exercise plans to manage PCOS effectively.")

def pcos_chatbot():
    print("Welcome to the PCOS detection chatbot!")
    
    while True:
        print("\nYou can enter your request (e.g., 'predict PCOS', 'get diet and exercise recommendations', or 'quit').")
        
        user_input = input("\nWhat would you like to do? ").strip().lower()

        # Check for PCOS prediction keywords
        if "predict" in user_input or "diagnosis" in user_input:
            print("\nPlease answer the following questions for PCOS prediction:")
            user_data = []
            blood_group_mapping = {
                'A+': 11, 'A-': 12,
                'B+': 13, 'B-': 14,
                'O+': 15, 'O-': 16,
                'AB+': 17, 'AB-': 18
            }

            for feature in X.columns:
                if 'blood group' in feature.lower():
                    while True:
                        blood_group = input("Enter your blood group (e.g., A+, O-): ").upper()
                        if blood_group in blood_group_mapping:
                            value = blood_group_mapping[blood_group]
                            break
                        else:
                            print("Invalid blood group entered. Please try again.")
                else:
                    while True:
                        try:
                            value = float(input(f"Enter value for {feature} (numeric input expected): "))
                            break
                        except ValueError:
                            print("Invalid input. Please enter a numeric value.")
                user_data.append(value)

            # Make prediction
            user_data_scaled = scaler.transform([user_data])
            prediction = model.predict(user_data_scaled)

            if prediction == 1:
                print("\nThe chatbot predicts that you might have PCOS.")
            else:
                print("\nThe chatbot predicts that you likely do not have PCOS. However, consult a doctor if you have concerns.")

        # Check for recommendation keywords
        elif "recommendation" in user_input or "recommend" in user_input:
            print("\nLet's collect more information for a personalized lifestyle plan.")
            busy_morning = input("Do you usually have a busy morning schedule? (yes/no): ").strip().lower()
            work_hours = int(input("On average, how many hours do you spend at work or school each day? (0-12): ").strip())
            exercise_freq = input("How often do you exercise each week? (never/1-2 times/3-5 times/daily): ").strip().lower()
            preferred_exercise_type = input("Do you prefer high-intensity or low-intensity exercises? (high/low): ").strip().lower()
            time_of_day_exercise = input("When do you prefer to exercise? (morning/evening/both): ").strip().lower()
            exercise_duration = int(input("How much time can you dedicate to each exercise session? (in minutes): ").strip())

            generate_diet_and_exercise_plan()

        # Exit the chatbot
        elif "quit" in user_input:
            print("\nThank you for using the PCOS chatbot. Goodbye!")
            break

        else:
            print("\nInvalid input. Please enter a valid command (e.g., 'predict PCOS', 'get diet and exercise recommendations', or 'quit').")

# Run the chatbot
pcos_chatbot()

Model Accuracy: 0.9354838709677419
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96        80
         1.0       0.82      0.69      0.75        13

    accuracy                           0.94        93
   macro avg       0.88      0.83      0.86        93
weighted avg       0.93      0.94      0.93        93

Welcome to the PCOS detection chatbot!

You can enter your request (e.g., 'predict PCOS', 'get diet and exercise recommendations', or 'quit').



What would you like to do?  Can you predict pcos for me?



Please answer the following questions for PCOS prediction:


Enter value for Age (in Years) (numeric input expected):  20
Enter value for Weight (in Kg) (numeric input expected):  64
Enter value for Height (in Cm / Feet) (numeric input expected):  153
Enter your blood group (e.g., A+, O-):  O+
Enter value for After how many months do you get your periods?
(select 1- if every month/regular) (numeric input expected):  2
Enter value for Have you gained weight recently? (numeric input expected):  0
Enter value for Do you have excessive body/facial hair growth ? (numeric input expected):  0
Enter value for Are you noticing skin darkening recently? (numeric input expected):  1
Enter value for Do have hair loss/hair thinning/baldness ? (numeric input expected):  1
Enter value for Do you have pimples/acne on your face/jawline ? (numeric input expected):  0
Enter value for Do you eat fast food regularly ? (numeric input expected):  1
Enter value for Do you exercise on a regular basis ? (numeric input expected):  0
Enter value for Do you experience mood s

/home/jovyan/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...


Let's collect more information for a personalized lifestyle plan.


Do you usually have a busy morning schedule? (yes/no):  yes
On average, how many hours do you spend at work or school each day? (0-12):  8
How often do you exercise each week? (never/1-2 times/3-5 times/daily):  never
Do you prefer high-intensity or low-intensity exercises? (high/low):  low
When do you prefer to exercise? (morning/evening/both):  both
How much time can you dedicate to each exercise session? (in minutes):  30



--- Personalized Weekly Diet Plan ---
+-----------+---------------------------------+-----------------------------------+-------------------+
| Days      | Breakfast                       | Lunch                             | Dinner            |
+===========+=================================+===================================+===================+
| Sunday    | Hibiscus Tea                    | Hibiscus Tea                      | Pongal            |
+-----------+---------------------------------+-----------------------------------+-------------------+
| Monday    | Chia Seed Pudding with Berries  | Sambar                            | Vegetable Sambar  |
+-----------+---------------------------------+-----------------------------------+-------------------+
| Tuesday   | Lentil Soup                     | Palak Dal                         | Green Smoothie    |
+-----------+---------------------------------+-----------------------------------+-------------------+
| Wednesday | Sweet Potat


What would you like to do?  quit



Thank you for using the PCOS chatbot. Goodbye!
